In [1]:
!pip install pymysql > /dev/null
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json
with open('mysql.json', 'r') as file:
  config_str = file.read()
config = json.loads(config_str)

In [3]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

# users table에 패스워드 넣기

## 패스워드 암호화
- Secure Hash Algorithm(SHA-256)

In [4]:
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- base64로 인코딩

In [5]:
import base64

base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [7]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [8]:
len(hashed_pwd)

44

- 패스워드를 생성하는 함수

In [9]:
import hashlib, base64

def gen_pwd(pwd):
  pwd_sha256 = hashlib.sha256(pwd.encode())
  hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
  return hashed_pwd

In [10]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기
- uid + 21

In [26]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [42]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
  uid = line[0]
  pwd = gen_pwd(uid+'21')
  cur.execute(sql_update, (pwd, uid))
conn.commit()

- email 추가 : uid@mc.com

In [43]:
sql_update = 'update users set email=%s where uid=%s;'
for line in results:
  uid = line[0]
  email = uid+'@mc.com'
  cur.execute(sql_update, (email, uid))
conn.commit()

## 로그인

- case1 : uid가 없는 경우

In [48]:
# result = Null

uid = 'fool'
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result


- case2 : 비밀번호가 틀린경우

In [49]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

- case3 : OK

In [51]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
  print('login')
else:
  print('incorrect')

incorrect


- 3가지 경우를 합침

In [53]:
uid, pwd = 'dj2y', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()

if result:
  db_pwd = result[1]
  hashed_pwd = gen_pwd(pwd)
  if db_pwd == hashed_pwd:
    print('login')
  else:
    print('incorrect')
else:
  print('Bad UID')

Bad UID


- 예외처리

In [54]:
uid, pwd = 'dj2y', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()

try:
  db_pwd = result[1]
  hashed_pwd = gen_pwd(pwd)
  if db_pwd == hashed_pwd:
    print('login')
  else:
    print('incorrect')
except:
  print('Bad UID')

Bad UID


In [55]:
cur.close()
conn.close()